# Story

Let's consider a clique of colleges from work and an gossip spreading between them. In our experiment we will simulate this phenomena.

Colleges communicate via social network and during face-to-face meetings. All of them know each other, but during pandemic people usually works remotely and our guys are no exception from the rule. Hence some of them don't go to bureau - we will simulate it by disconnecting nodes in the 'work' layer. Moreover it's obvious that Internet has bigger impact in the field of information spreading than real interactions. By that reason in our model transitions for social network will be higher weighted than for work layer.

In [ ]:
from typing import Any

import matplotlib.pyplot as plt
import networkx as nx

from network_diffusion import (
    MultilayerNetwork,
    MultiSpreading,
    PropagationModel,
)

# set deefault values for matplotlib
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 150 

# Create graph

We will create multilayer graph with two layers: work_layer, twtr_layer First will be less dense than second - this operation will simulate less intensive interaction in the bureau in comparasion to the social life.

In [ ]:
# initialise layers
work_layer = nx.karate_club_graph()
nodes_to_remove = [
    (32, 33), 
    (4, 10), 
    (2, 3), 
    (3, 7),
    (23, 27),
    (20, 33),
    (0, 10), 
    (22, 33), 
    (28, 33), 
    (23, 29),
    (2, 7),
    (29, 33),
    (0, 8), 
    (26, 33),
    (1, 7), 
    (0, 1), 
    (15, 33),
    (0, 21), 
    (3, 13), 
    (1, 19),
    (4, 6), 
    (30, 32),
]
work_layer.remove_edges_from(nodes_to_remove)
twtr_layer = nx.karate_club_graph()

# initialise multilayer graph
network = MultilayerNetwork()
network.load_layers_nx([work_layer, twtr_layer], ["inf_work", "inf_twtr"])

In [ ]:
# auxiliary cell - some description of the network
network.describe()

layout = nx.spring_layout(twtr_layer)
fig, ax = plt.subplots(nrows=1, ncols=2)
nx.draw(work_layer, with_labels=True, pos=layout, ax=ax[0])
ax[0].set_title('Work layer')
nx.draw(twtr_layer, with_labels=True, pos=layout, ax=ax[1])
ax[1].set_title('Twtr layer')
plt.show()

# Define propagation model

Now let's create a propagation model. In our story we distinguished one phenomena (gossip) which is beinng spreaden in two environments: work and social communicator. Relation between person (node) and phenomena (gossip) apears in two states: aware (A) and unaware (U). In these condition each individual can get to know about the gossip in the work with some pseudo-probability* x and via the social communicatior with pseeudo-probability y, like in picture below:

<img src="aux/model_real.png">

In fact, network-diffusion allows to model phenomenas like this. To do it we must however write them in a way unnderstandable to the library. For our case will do it like that:

<img src="aux/model_nd.png">

In another words we define a model as usual, but we set up pseudo-probability to 1 in two trasitions:
* Unaware(Social Network), Aware(Work) -> Aware(Social Network), Aware(Work); pseudo-probability = 0.1
* Aware(Social Network), Unaware(Work) -> Aware(Social Network), Aware(Work); pseudo-probability = 0.05

In [ ]:
# initialise propagation model
model = PropagationModel()
phenomenas = [["U", "A"], ["U", "A"]]
for l, p in zip(network.get_layer_names(), phenomenas):
    model.add(l, p)

# define weights
w_background = 0
w_x = 0.05
w_y = 0.1
 
# set possible transitions with weights
model.compile(background_weight=w_background)

model.set_transition_fast("inf_work.U", "inf_work.A", (["inf_twtr.U"]), w_x)
model.set_transition_fast("inf_work.U", "inf_work.A", (["inf_twtr.A"]), 1)

model.set_transition_fast("inf_twtr.U", "inf_twtr.A", (["inf_work.U"]), w_y)
model.set_transition_fast("inf_twtr.U", "inf_twtr.A", (["inf_work.A"]), 1)

In [ ]:
# description of the network
model.describe()

The last step in setting up the simulation is a definition of experiment. To do it we have to connect model and network, and initialise states of the nodes. We can initialise theem randomly or manually by assigning status of certain nodes.

As starting parameters of simulation we will set:
* in layer Social Network 32 nodes unaware and 2 aware of gossip
* in layer Work 33 nodes unaware and 1 aware of gossip

in two ways

In [ ]:
# option A - setting up states for randomly choosen nodes
experiment = MultiSpreading(model, network)
experiment.set_initial_states({"inf_twtr": (32, 2), "inf_work": (33, 1)})

In [ ]:
# option B - setting up states for manually choosen nodes
def set_node_state(
    experiment: MultiSpreading, layer_name: str, node_namee: Any, state: str
) -> None:
    node_idx = list(experiment._network.layers[layer_name].nodes).index(node_namee)
    experiment._network.layers[layer_name].nodes[node_idx]["status"] = state

experiment = MultiSpreading(model, network)
experiment.set_initial_states({"inf_twtr": (34, 0), "inf_work": (34, 0)})

set_node_state(experiment, "inf_twtr", 2, "A")
set_node_state(experiment, "inf_twtr", 5, "A")

set_node_state(experiment, "inf_work", 2, "A")

In [ ]:
# run experiment
init_status = experiment._network.get_nodes_states()
logs = experiment.perform_propagation(n_epochs=20)
final_status = experiment._network.get_nodes_states()

print(f"State of nodes before experiment {init_status}")
print(f"State of nodes after  experiment {final_status}")

In [ ]:
# show logs
logs.report(to_file=False, path=None, visualisation=True)